<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">
           <a href="https://bokeh.org/">
           <img
               src="assets/bokeh-transparent.png"
               style="width:50px"
           >
           </a>
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>09 More plot types</h2></div>

In [ ]:
# activate notebook output
from bokeh.io import output_notebook

output_notebook()

# load tutorial data set
import sys

sys.path.append("../data")
from carriers_data import CarrierDataSet

data = CarrierDataSet()

This chapter introduces additional plot types:
- Map plots
- Wedge plots (donut and pie charts)
- [TBD] Subplots

### Map plots

You can use Bokeh to visualize geographical data on maps.

To process data with geo information, Bokeh includes the GeoJSONDataSource. The
GeoJSONDataSource works similar to the ColumnDataSource.

You can use [GeoJSON](https://geojson.org/) data such as points, lines, and polygons
(called Patches in Bokeh) together with other data from a ColumnDataSource.



In [ ]:
import geopandas as gpd

from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, LinearColorMapper
from bokeh.palettes import Cividis11

states_gdf = gpd.read_file("../data/us-states.geojson")
states_gdf = states_gdf.join(data.get_states_routes_df(), on=states_gdf["Name"])

MAP_SELECTIONS = ["origin", "destination"]
selected_map = MAP_SELECTIONS[0]  # render map either based on origin or destination

TOOLTIPS = [("State", "@Name")]
TOOLTIPS.append(("# of routes departing from here", "@origin{(0,0)}"))

map_plot = figure(
    height=300,
    width=500,
    tooltips=TOOLTIPS,
    title="Number of routes with a state as its origin (all domestic carriers)",
    x_axis_location=None,
    y_axis_location=None,
)

map_plot.grid.grid_line_color = None

mapper = LinearColorMapper(
    palette=list(Cividis11),
    low=states_gdf[selected_map].min(),
    high=states_gdf[selected_map].max(),
)

geo_source = GeoJSONDataSource(geojson=states_gdf.to_json())

us = map_plot.patches(
    xs="xs",
    ys="ys",
    fill_color=dict(field=selected_map, transform=mapper),
    source=geo_source,
    line_color="#333344",
    line_width=1,
)

map_plot.x_range.renderers = [us]
map_plot.y_range.renderers = [us]

show(map_plot)

Also: map xyz providers google maps

#### Wedge plots

[TBD]

In [ ]:
## [TBD] simplify example, remove Tabs, add more annotations

import copy
from math import pi

from bokeh.plotting import figure, show
from bokeh.models import TabPanel, Tabs
from bokeh.palettes import Viridis
from bokeh.transform import cumsum

# create list of colors (Spectral10 plus gray for "other")
colors = list(Viridis[10])
colors.append("#808080")


def truncate_names(airline_name, max_len=25):
    """Truncate long airline names to max_len"""
    if len(airline_name) > max_len:
        airline_name = airline_name[: max_len - 3] + "..."
    return airline_name


def create_dfs(df, categories):
    """Create dict of dfs for passengers, freight, and mail"""
    dfs = {}
    for category in categories:
        # create copy of df for current category
        category_df = df
        # sort dataframe by current category
        category_df = category_df.sort_values(category, ascending=False)
        category_df.reset_index(inplace=True, drop=True)
        # remove rows that are not the current category
        remove_columns = copy.deepcopy(categories)
        remove_columns.remove(category)
        category_df.drop(columns=remove_columns, inplace=True)
        # sum values for "others" (all carriers not in top 10)
        top_ten_by_category = category_df.iloc[:10]["unique_carrier_name"]
        other_sum = category_df[~category_df["unique_carrier_name"].isin(top_ten_by_category)][category].sum()
        # create dataframe for top 10 of current category plus others
        category_df = category_df[category_df["unique_carrier_name"].isin(top_ten_by_category)]
        category_df.loc[len(category_df.index)] = ["Others", other_sum]
        # add column with annular wedge angles
        category_df["angle"] = category_df[category] / category_df[category].sum() * 2 * pi
        # assign colors to carriers
        category_df["color"] = colors
        # truncate long carrier names
        category_df["unique_carrier_name"] = category_df["unique_carrier_name"].apply(truncate_names, args=(25,))
        # add category dataframe to dict of dataframes
        dfs[category] = category_df

    return dfs


def create_annular_wedge(df_dict, category):
    """Create annular wedge plots for passengers, freight, and mail"""
    TOOLTIPS = [
        ("Carrier", "@unique_carrier_name"),
        (category.capitalize(), f"@{category}{{(0,0)}}"),
    ]

    annular_plot = figure(
        height=300,
        toolbar_location=None,
        outline_line_color=None,
        sizing_mode="scale_width",
        name="region",
        x_range=(-0.66, 1),
        title=f"Top ten carriers by {category}",
        tooltips=TOOLTIPS,
    )

    annular_plot.annular_wedge(
        x=0,
        y=0,
        inner_radius=0.2,
        outer_radius=0.4,
        start_angle=cumsum("angle", include_zero=True),
        end_angle=cumsum("angle"),
        line_color="white",
        fill_color="color",
        legend_field="unique_carrier_name",
        source=df_dict[category],
    )

    annular_plot.axis.visible = False
    annular_plot.grid.grid_line_color = None
    annular_plot.legend.spacing = 1
    annular_plot.legend.label_text_font_size = "0.8em"

    return annular_plot


# list of categories to consider
categories = data.measurements

# create dataframes for each category
dfs = create_dfs(data.get_carriers_df(), categories)

# create tabs with annular wedges for each category
tabs = []
for category in categories:
    tabs.append(TabPanel(child=create_annular_wedge(dfs, category), title=category.capitalize()))

# display all plots as tabs
annular_wedge_tabs = Tabs(tabs=tabs, sizing_mode="scale_both")

show(annular_wedge_tabs)

# Next section

[TBD, placeholder]